In [1]:
import os
import sys
import pandas as pd

sys.path.append(os.path.join(os.path.expanduser('~'), 'Projects', 'TATSSI'))

# TATSSI Time Series Generator
from TATSSI.time_series.generator import Generator
# Notebook helpers
from TATSSI.notebooks.helpers.utils import PlotTimeSeries
# Catalogue
from TATSSI.qa.EOS.catalogue import Catalogue

from beakerx import TableDisplay
from IPython.display import display
from ipywidgets import Select, SelectMultiple
from ipywidgets import Button, HBox, VBox
from ipywidgets import Layout

# Set data directory
DataDir = '/home/glopez/Projects/TATSSI/data/MOD13A2.006'
# Set product, it must match product name in TATSSI catalogue
product, version = 'MOD13A2', '006'
product_and_version = f'{product}.{version}'

# Create the time series generator object
tsg = Generator(source_dir = DataDir, product = product_and_version)

# Load time series data!
ts = tsg.load_time_series()

In [2]:
catalogue = Catalogue()
qa_defs = catalogue.get_qa_definition(product=product, version=version)

decimals_list = []
for qa_value in qa_defs[1].Value:
    decimals_list.append(int(bin(qa_value)[2:]))

qa_defs[1]['Value'] = decimals_list

In [46]:
qa_defs[1]

,ProductAndVersion,QualityLayer,Name,Value,Description,Acceptable,Length
0,MOD13A2.006,_1_km_16_days_VI_Quality,MODLAND,0,"VI produced, good quality",True,2
1,MOD13A2.006,_1_km_16_days_VI_Quality,MODLAND,1,"VI produced, but check other QA",False,2
2,MOD13A2.006,_1_km_16_days_VI_Quality,MODLAND,10,"Pixel produced, but most probably cloudy",False,2
3,MOD13A2.006,_1_km_16_days_VI_Quality,MODLAND,11,Pixel not produced due to other reasons than c...,False,2
4,MOD13A2.006,_1_km_16_days_VI_Quality,VI Usefulness,0,Highest quality,None,4
5,MOD13A2.006,_1_km_16_days_VI_Quality,VI Usefulness,1,Lower quality,None,4
6,MOD13A2.006,_1_km_16_days_VI_Quality,VI Usefulness,10,Decreasing quality (0010),None,4
7,MOD13A2.006,_1_km_16_days_VI_Quality,VI Usefulness,11,Decreasing quality (0011),None,4
8,MOD13A2.006,_1_km_16_days_VI_Quality,VI Usefulness,100,Decreasing quality (0100),None,4
9,MOD13A2.006,_1_km_16_days_VI_Quality,VI Usefulness,101,Decreasing quality (0101),None,4


In [89]:
qa_def = qa_defs[1]
qa_flags = qa_def.Name.unique()

global user_qa_selection
user_qa_selection = dict((element, '') for element in qa_flags)
# Fill default selection
for i, selection in enumerate(user_qa_selection):
    user_qa_selection[selection] = tuple([qa_def[qa_def.Name == selection].Description.tolist()[0]])

qa_layer = Select(
    options=qa_flags,
    value=qa_flags[0],
    rows=len(qa_flags),
    description='QA flag:',
    disabled=False
)

qa_description = SelectMultiple(
    options=qa_def[qa_def.Name == qa_layer.value].Description.tolist(),
    value=[qa_def[qa_def.Name == qa_layer.value].Description.tolist()[0]],
    rows=len(qa_def[qa_def.Name == qa_layer.value].Description.tolist()),
    description='Description',
    disabled=False,
    layout={'width': '500px'}
)

def on_qa_layer_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        qa_layer_value = change.owner.value
        
        # Get user selection before changing qa description
        tmp_selection = user_qa_selection[qa_layer_value]

        _options = qa_def[qa_def.Name == qa_layer_value].Description.tolist()
        qa_description.options = _options

        #print(qa_layer_value, user_qa_selection[qa_layer_value])
        #if user_qa_selection[qa_layer_value] == '' or \
        #   user_qa_selection[qa_layer_value] == '()':
        #    # Initial value
        #    qa_description.value = [_options[0]]
        #else:
        #    # User selection
        #    qa_description.value = user_qa_options[qa_layer_value]
        
        qa_description.rows = len(_options)
        qa_description.value = tmp_selection
    
qa_layer.observe(on_qa_layer_change)

def on_qa_description_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        user_qa_selection[qa_layer.value] = qa_description.value

qa_description.observe(on_qa_description_change)

def select_all_qa(b):
    for i, selection in enumerate(user_qa_selection):
        user_qa_selection[selection] = tuple([qa_def[qa_def.Name == selection].Description.tolist()])
        #qa_layer.value= qa_flags[0]

# Select all button
select_all = Button(
    description = 'Select ALL',
    layout={'positioning' : 'right'}
)
select_all.on_click(select_all_qa)

left_box = VBox([qa_description])
right_box = VBox([qa_description])
HBox(
    [qa_layer, right_box, select_all],
    layout={'height': '350px',
            'width' : '99%'}
)

#display(HBox)
#idgetLink = widgets.jslink((qa_layer, 'value'), (floatSlider, 'value'))

In [91]:
aa

{'Adjacent cloud detected': ('No', 'Yes'),
 'Aerosol Quantity': ('Climatology', 'Low', 'Average', 'High'),
 'Atmosphere BRDF Correction': ('No', 'Yes'),
 'Land/Water Mask': ('Shallow ocean',
  'Land (Nothing else but land)',
  'Ocean coastlines and lake shorelines',
  'Shallow inland water',
  'Ephemeral water',
  'Deep inland water',
  'Moderate or continental ocean',
  'Deep ocean'),
 'MODLAND': (),
 'Mixed Clouds': ('No', 'Yes'),
 'Possible shadow': ('No', 'Yes'),
 'Possible snow/ice': ('No', 'Yes'),
 'VI Usefulness': ('Highest quality',
  'Lower quality',
  'Decreasing quality (0010)',
  'Decreasing quality (0011)',
  'Decreasing quality (0100)',
  'Decreasing quality (0101)',
  'Decreasing quality (0110)',
  'Decreasing quality (0111)',
  'Decreasing quality (1000)',
  'Decreasing quality (1001)',
  'Decreasing quality (1010)',
  'Decreasing quality (1011)',
  'Lowest quality',
  'Quality so low that it is not useful',
  'L1B data faulty',
  'Not useful for any other reason/not pr

In [92]:
user_qa_selection

{'Adjacent cloud detected': (['No', 'Yes'],),
 'Aerosol Quantity': (['Climatology', 'Low', 'Average', 'High'],),
 'Atmosphere BRDF Correction': (['No', 'Yes'],),
 'Land/Water Mask': (['Shallow ocean',
   'Land (Nothing else but land)',
   'Ocean coastlines and lake shorelines',
   'Shallow inland water',
   'Ephemeral water',
   'Deep inland water',
   'Moderate or continental ocean',
   'Deep ocean'],),
 'MODLAND': (['VI produced, good quality',
   'VI produced, but check other QA',
   'Pixel produced, but most probably cloudy',
   'Pixel not produced due to other reasons than clouds'],),
 'Mixed Clouds': (['No', 'Yes'],),
 'Possible shadow': (['No', 'Yes'],),
 'Possible snow/ice': (['No', 'Yes'],),
 'VI Usefulness': (['Highest quality',
   'Lower quality',
   'Decreasing quality (0010)',
   'Decreasing quality (0011)',
   'Decreasing quality (0100)',
   'Decreasing quality (0101)',
   'Decreasing quality (0110)',
   'Decreasing quality (0111)',
   'Decreasing quality (1000)',
   'Dec

In [ ]:
print(ts.qa._fields)
ts.qa.qa_1_km_16_days_VI_Quality

In [ ]:
# Time series of EVI
p = PlotTimeSeries()
p.plot(ts.data._1_km_16_days_EVI, ts.qa.qa_1_km_16_days_VI_Quality.Land_Water_Mask)